In [1]:
import numpy as np
import pandas as pd
import keras

Using TensorFlow backend.


In [2]:
kernel_init_method = keras.initializers.glorot_uniform(seed=0)

In [3]:
def identity_block(X, f, filters):
    """
    Implementation of the identity block as defined in Figure 4
    
    Arguments:
    X -- input tensor of shape (m, n_H_prev, n_W_prev, n_C_prev)
    f -- integer, specifying the shape of the middle CONV's window for the main path
    filters -- python list of integers, defining the number of filters in the CONV layers of the main path
    stage -- integer, used to name the layers, depending on their position in the network
    block -- string/character, used to name the layers, depending on their position in the network
    
    Returns:
    X -- output of the identity block, tensor of shape (n_H, n_W, n_C)
    """
    F1, F2, F3 = filters
    X_skip = X # Skip Connection

    # Define the block
    X = keras.layers.Conv2D(F1, kernel_size = (1, 1), strides=(1, 1), padding = 'valid', kernel_initializer=kernel_init_method)(X)
    X = keras.layers.BatchNormalization(axis = 3)(X)
    X = keras.layers.Activation('relu')(X)

    # Second stage
    X = keras.layers.Conv2D(F2, kernel_size = (f, f), strides=(1, 1), padding = 'same', kernel_initializer=kernel_init_method)(X)
    X = keras.layers.BatchNormalization(axis = 3)(X)
    X = keras.layers.Activation('relu')(X)

    #Third stage
    X = keras.layers.Conv2D(F3, kernel_size = (1, 1), strides=(1, 1), padding = 'valid', kernel_initializer=kernel_init_method)(X)
    X = keras.layers.BatchNormalization(axis = 3)(X)

    # Add skip connection and perform ReLU
    X = keras.layers.Add()([X_skip, X])
    X = keras.layers.Activation('relu')(X)

    return X

In [4]:
def convolutional_block(X, f, filters, s = 2):
    """
    Implementation of the convolutional block as defined in Figure 4
    
    Arguments:
    X -- input tensor of shape (m, n_H_prev, n_W_prev, n_C_prev)
    f -- integer, specifying the shape of the middle CONV's window for the main path
    filters -- python list of integers, defining the number of filters in the CONV layers of the main path
    stage -- integer, used to name the layers, depending on their position in the network
    block -- string/character, used to name the layers, depending on their position in the network
    s -- Integer, specifying the stride to be used
    
    Returns:
    X -- output of the convolutional block, tensor of shape (n_H, n_W, n_C)
    """
    X_skip = X
    F1, F2, F3 = filters

    # First stage
    X = keras.layers.Conv2D(F1, kernel_size = (1, 1), strides= (s, s), padding = 'valid', kernel_initializer=kernel_init_method)(X)
    X = keras.layers.BatchNormalization(axis = 3)(X)
    X = keras.layers.Activation('relu')(X)

    # Second stage
    X = keras.layers.Conv2D(F2, kernel_size = (f, f), strides= (1, 1), padding = 'same', kernel_initializer=kernel_init_method)(X)
    X = keras.layers.BatchNormalization(axis = 3)(X)
    X = keras.layers.Activation('relu')(X)

    # Third stage
    X = keras.layers.Conv2D(F3, kernel_size = (1, 1), strides= (1, 1), padding = 'valid', kernel_initializer=kernel_init_method)(X)
    X = keras.layers.BatchNormalization(axis = 3)(X)
    
    # Skip path
    X_skip = keras.layers.Conv2D(F3, kernel_size=(1, 1), strides = (s, s), padding = 'valid', kernel_initializer=kernel_init_method)(X_skip)
    X_skip = keras.layers.BatchNormalization(axis = 3)(X_skip)
    X = keras.layers.Add()([X_skip, X])
    X = keras.layers.Activation('relu')(X)

    return X

In [5]:
def ResNet(input_shape = (64, 64, 3), classes = 6):
  """
    CONV2D -> BATCHNORM -> RELU -> MAXPOOL -> CONVBLOCK -> IDBLOCK*2 -> CONVBLOCK -> IDBLOCK*3
    -> CONVBLOCK -> IDBLOCK*5 -> CONVBLOCK -> IDBLOCK*2 -> AVGPOOL -> TOPLAYER

    Arguments:
    input_shape -- shape of the images of the dataset
    classes -- integer, number of classes

    Returns:
    model -- a Model() instance in Keras
  """
  X_Input = keras.layers.Input(shape = input_shape) #TODO Add Batch numbers
  X = keras.layers.ZeroPadding2D(padding = (3, 3)) (X_Input)
  # X = X_Input

  #Stage 1
  X = keras.layers.Conv2D(filters = 32, kernel_size = (3, 3), strides = (2, 2), padding = 'same', kernel_initializer=kernel_init_method)(X)
  X = keras.layers.BatchNormalization(axis = 3)(X)
  X = keras.layers.Activation('relu')(X)
  X = keras.layers.MaxPool2D(pool_size = (2, 2), strides = (2, 2))(X)

  # Stage 2
  X = convolutional_block(X, f = 3, filters = [32, 32, 128], s = 1)
  X = identity_block(X, f = 3, filters = [32, 32, 128])
  X = identity_block(X, f = 3, filters = [32, 32, 128])

  # Stage 3
  X = convolutional_block(X, f = 3, filters = [64, 64, 256], s = 2)
  X = identity_block(X, f = 3, filters = [64, 64, 256])
  X = identity_block(X, f = 3, filters = [64, 64, 256])
  X = identity_block(X, f = 3, filters = [64, 64, 256])

  # # # Stage 4
  # X = convolutional_block(X, f = 3, filters = [256, 256, 1024], s = 2)
  # X = identity_block(X, f = 3, filters = [256, 256, 1024])
  # X = identity_block(X, f = 3, filters = [256, 256, 1024])
  # X = identity_block(X, f = 3, filters = [256, 256, 1024])
  # X = identity_block(X, f = 3, filters = [256, 256, 1024])
  # X = identity_block(X, f = 3, filters = [256, 256, 1024])

  # # # Stage 5
  # X = convolutional_block(X, f = 3, filters = [512, 512, 2048], s = 2)
  # X = identity_block(X, f = 3, filters = [512, 512, 2048])
  # X = identity_block(X, f = 3, filters = [512, 512, 2048])

  # Stage FC
  X = keras.layers.AvgPool2D(pool_size=(2, 2))(X)
  X = keras.layers.Flatten()(X)
  X = keras.layers.Dense(classes, activation='softmax', kernel_initializer = kernel_init_method)(X)

  model = keras.models.Model(inputs = X_Input, outputs = X, name = 'ResNet-50')

  return model


In [6]:
model = ResNet(input_shape = (32, 32, 3), classes = 100)
print(model.summary())

Model: "ResNet-50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
zero_padding2d_1 (ZeroPadding2D (None, 38, 38, 3)    0           input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 19, 19, 32)   896         zero_padding2d_1[0][0]           
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 19, 19, 32)   128         conv2d_1[0][0]                   
__________________________________________________________________________________________

In [18]:
#Train the model
model.fit(
    ds_train.as_numpy_iterator(),
    epochs=6,
    validation_data=ds_test.as_numpy_iterator(),
    verbose = 1
)

AttributeError: ignored

In [8]:
import tensorflow.compat.v2 as tf
import tensorflow_datasets as tfds

# Construct a tf.data.Dataset
(ds_train, ds_test), ds_info = tfds.load('cifar100', split=['train', 'test'], shuffle_files=True, as_supervised=True, with_info= True)
print(ds_info)

tfds.core.DatasetInfo(
    name='cifar100',
    version=3.0.0,
    description='This dataset is just like the CIFAR-10, except it has 100 classes containing 600 images each. There are 500 training images and 100 testing images per class. The 100 classes in the CIFAR-100 are grouped into 20 superclasses. Each image comes with a "fine" label (the class to which it belongs) and a "coarse" label (the superclass to which it belongs).',
    homepage='https://www.cs.toronto.edu/~kriz/cifar.html',
    features=FeaturesDict({
        'coarse_label': ClassLabel(shape=(), dtype=tf.int64, num_classes=20),
        'image': Image(shape=(32, 32, 3), dtype=tf.uint8),
        'label': ClassLabel(shape=(), dtype=tf.int64, num_classes=100),
    }),
    total_num_examples=60000,
    splits={
        'test': 10000,
        'train': 50000,
    },
    supervised_keys=('image', 'label'),
    citation="""@TECHREPORT{Krizhevsky09learningmultiple,
        author = {Alex Krizhevsky},
        title = {Learning mul

In [9]:
# Training pipeline
def normalize_img(image, label):
  """Normalizes images: `uint8` -> `float32`."""
  return tf.cast(image, tf.float32) / 255., label

ds_train = ds_train.map(
    normalize_img, num_parallel_calls=tf.data.experimental.AUTOTUNE)
ds_train = ds_train.cache()
ds_train = ds_train.shuffle(ds_info.splits['train'].num_examples)
ds_train = ds_train.batch(128)
ds_train = ds_train.prefetch(tf.data.experimental.AUTOTUNE)

# Evaluation pipeline
def normalize_img(image, label):
  """Normalizes images: `uint8` -> `float32`."""
  return tf.cast(image, tf.float32) / 255., label

ds_train = ds_train.map(
    normalize_img, num_parallel_calls=tf.data.experimental.AUTOTUNE)
ds_train = ds_train.cache()
ds_train = ds_train.shuffle(ds_info.splits['train'].num_examples)
ds_train = ds_train.batch(128)
ds_train = ds_train.prefetch(tf.data.experimental.AUTOTUNE)


In [12]:
print(ds_train.as_numpy_iterator())